## Random forest Classifier with pipeline and hyper parametr tunning

In [2]:
import seaborn as sns


In [3]:
df=sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [5]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [7]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [8]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['time']=encoder.fit_transform(df['time'])

In [9]:
df['time'].unique()

array([0, 1])

In [10]:
## independet and dependent features

x=df.drop(labels=['time'],axis=1)#independent feature
y=df.time#dependent feature

In [11]:
x.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [12]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: time, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer   # Handle missing values nije theke e dkehe nay
from sklearn.preprocessing import StandardScaler # feature Scalling 
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer



In [15]:
categorical_columns=['sex','smoker','day']
numerical_columns=['total_bill','tip','size']


## for feature engineering 

In [16]:
# Numerical pipeline
numerical_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),  # missing values 
        ('scaler', StandardScaler())  # feature scaling
    ]
)

# CATEGORICAL PIPELINE
categorical_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),  # missing values
        ('onehotencoder', OneHotEncoder())  # categorical feature to numerical feature
    ]
)


In [17]:
preproccessor=ColumnTransformer([
    ('numerical_pipeline',numerical_pipeline,numerical_columns),
    ('categorical_pipeline',categorical_pipeline,categorical_columns)
])


In [18]:
x_train=preproccessor.fit_transform(x_train)
x_test=preproccessor.transform(x_test)

## Model training automation

In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor

In [36]:
models={
    'Random forest':RandomForestClassifier(),
    'Logistic regression':LogisticRegression(),
    'Decision TreeDekh Classifier':DecisionTreeClassifier(),
    'Decision Tree Regressor':DecisionTreeRegressor()
}


In [40]:
from sklearn.metrics import accuracy_score,r2_score,precision_score


In [44]:
def evaluate_model(X_train,y_train,X_test,y_test,models):

    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        # Train model
        model.fit(X_train,y_train)



        # Predict Testing data
        y_test_pred =model.predict(X_test)

        # Get accuracy for test data prediction

        test_model_score1 = accuracy_score(y_test,y_test_pred)
        
         # Get R2 for test data prediction

        test_model_score2 = r2_score(y_test,y_test_pred)
        
         # Get precisiom for test data prediction

        test_model_score3 = precision_score(y_test,y_test_pred)

        report[list(models.keys())[i]] =  (test_model_score1,test_model_score2,test_model_score3)



    return report


In [45]:
evaluate_model(x_train,y_train,x_test,y_test,models)

{'Random forest': (0.9591836734693877, 0.807843137254902, 0.8823529411764706),
 'Logistic regression': (1.0, 1.0, 1.0),
 'Decision TreeDekh Classifier': (0.9387755102040817,
  0.711764705882353,
  0.9285714285714286),
 'Decision Tree Regressor': (0.9387755102040817,
  0.711764705882353,
  0.9285714285714286)}

In [48]:
#  best_model_score=max(sorted(model_report.values()))

In [49]:
classifier=RandomForestClassifier()

In [50]:
## Hyperparameter tunning

In [51]:
## Hypeparameter Tuning
params={'max_depth':[3,5,10,None],
              'n_estimators':[100,200,300],
               'criterion':['gini','entropy']
              }

In [52]:
from sklearn.model_selection import RandomizedSearchCV

In [54]:
cv=RandomizedSearchCV(classifier,param_distributions=params,scoring='accuracy',cv=5,verbose=3)

In [56]:
cv.fit(x_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=10, n_estimators=300;, score=0.974 total time=   0.5s
[CV 2/5] END criterion=entropy, max_depth=10, n_estimators=300;, score=0.923 total time=   0.5s
[CV 3/5] END criterion=entropy, max_depth=10, n_estimators=300;, score=1.000 total time=   0.5s
[CV 4/5] END criterion=entropy, max_depth=10, n_estimators=300;, score=0.949 total time=   0.5s
[CV 5/5] END criterion=entropy, max_depth=10, n_estimators=300;, score=0.923 total time=   0.5s
[CV 1/5] END criterion=gini, max_depth=3, n_estimators=100;, score=0.974 total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=3, n_estimators=100;, score=0.949 total time=   0.2s
[CV 3/5] END criterion=gini, max_depth=3, n_estimators=100;, score=0.974 total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=3, n_estimators=100;, score=0.923 total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=3, n_estimators=100;, score=0.923 total time=   0.2s

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [59]:
cv.best_params_

{'n_estimators': 200, 'max_depth': 10, 'criterion': 'entropy'}